## Node Similarity in Graphs

Node similarity is an important task in graph analysis, many node similarity approaches have been proposed 
in the literature. In this problem you are asked to compute node similarity using the following approaches 
for a Drug-Gene graph available at https://snap.stanford.edu/biodata/datasets/10002/10002-ChG-Miner.html 
(click on link ChG-Miner_miner-chem-gene.tsv.gz) to download the graph and identify the top 10 drug-drug pairs 
i.e. pairs of drugs that are similar.

1. Compute pairwise similarity for drugs node pairs (i.e. similarity between nodes u and v 
where both are drugs nodes) based on common neighbors. You may use one of the well known methods 
such as Jaccard, cosine etc. similarity measures.
2. Compute pairwise node similarity using the Sim-rank algorithm 
Compare and contrast the results using the two similarity measures (Jaccard/cosine etc. and Sim-Rank). 
Also try to interpret the similarity results from the two similarity measures.

Note: Node similarity using the Sim-Rank approach is available in NetworkX library. 


References:

https://networkx.org/documentation/networkx-1.7/tutorial/tutorial.html

http://snap.stanford.edu/snappy/index.html

In [1]:
import gzip
import shutil

from pathlib import Path
import pandas as pd
import networkx as net
import matplotlib.pyplot as plt
import operator


In [2]:
with gzip.open('ChG-Miner_miner-chem-gene.tsv.gz') as f:
    drugs = pd.read_csv(f, delimiter='\t')

drugs

,#Drug,Gene
0,DB00357,P05108
1,DB02721,P00325
2,DB00773,P23219
3,DB07138,Q16539
4,DB08136,P24941
...,...,...
15134,DB01215,P47870
15135,DB06089,P51787
15136,DB01614,P21728
15137,DB00582,P08684


In [3]:
list(drugs.columns)

['#Drug', 'Gene']

In [4]:
# G = net.Graph()
# # Add edges and edge attributes
# for i, row in drugs.iterrows():
#     G.add_edge(row[0], row[1])
    
G = net.from_pandas_edgelist(drugs, '#Drug', 'Gene')

In [ ]:
list(G.edges(data=True))

In [ ]:
list(net.connected_components(G))

In [ ]:
plt.figure(figsize=(350, 350))
# nx.draw(G, node_size=6, node_color='blue')
net.draw_networkx(G, node_size=6, node_color='blue')
plt.title('Graph Representation of Drug-target interaction Network', size=15)
plt.axis('off')
plt.savefig("drugs1.png")
plt.show()

In [ ]:
for components in list(net.connected_components(G)):
    print(f"Component size : {len(components)}")
    if len(components) > 100:
        H = G.subgraph(components)
    elif len(components) > 20:
        I = G.subgraph(components)
    elif len(components) > 10:
        J = G.subgraph(components)
    else:
        print(components)

#### Identify the top 10 drug-drug pairs i.e. pairs of drugs that are similar.

In [49]:
drugs_rank = net.pagerank_numpy(G, alpha=0.9)

In [50]:
top20 = dict(sorted(drugs_rank.items(), key=operator.itemgetter(1), reverse=True)[:20])    
top20

{'P08684': 0.011778982280539778,
 'P24941': 0.006749777697606485,
 'DB00157': 0.005983203977902857,
 'P10635': 0.005862599733219345,
 'P11712': 0.00547986649043369,
 'P07477': 0.00520074289511515,
 'P05177': 0.005120762238715825,
 'P00734': 0.005030194441549057,
 'P33261': 0.00405132207619656,
 'P00918': 0.0038840737240329135,
 'P10632': 0.003510919710299439,
 'P18031': 0.003400314981254162,
 'P20815': 0.0032962511961500845,
 'Q16539': 0.0029822752268185543,
 'P27487': 0.0028311179318616783,
 'P11217': 0.0026183723691104283,
 'P07900': 0.0025783289781575983,
 'P05181': 0.0023945604309667124,
 'P20813': 0.0023498365277685256,
 'DB00142': 0.002340918552640535}

In [51]:
top_nodes = list(top20.keys())
sub_graph = [(top_nodes[i], top_nodes[i+1]) for i in range(0, 11)]

#### Compute pairwise similarity for drugs node pairs (i.e. similarity between nodes u and v where both are drugs nodes) based on common neighbors. You may use one of the well known methods such as Jaccard, cosine etc. similarity measures.

In [54]:
preds = net.jaccard_coefficient(G, sub_graph)

In [55]:
for u, v, p in preds:
    print(f"({u}, {v}) -> {p:.8f}")

(P08684, P24941) -> 0.00138889
(P24941, DB00157) -> 0.00000000
(DB00157, P10635) -> 0.00000000
(P10635, P11712) -> 0.29237288
(P11712, P07477) -> 0.00000000
(P07477, P05177) -> 0.00000000
(P05177, P00734) -> 0.00274725
(P00734, P33261) -> 0.00000000
(P33261, P00918) -> 0.00943396
(P00918, P10632) -> 0.00000000
(P10632, P18031) -> 0.00000000


#### Considering a sub-graph "I" since sim_rank is taking very long to compute

In [40]:
preds = net.jaccard_coefficient(I, sub_graph)

In [41]:
for u, v, p in preds:
    print(f"({u}, {v}) -> {p:.8f}")

(P56817, DB07415) -> 0.00000000
(DB07415, DB07281) -> 1.00000000
(DB07281, DB07573) -> 1.00000000
(DB07573, DB07519) -> 1.00000000
(DB07519, DB08929) -> 1.00000000
(DB08929, DB07284) -> 1.00000000
(DB07284, DB07535) -> 1.00000000
(DB07535, DB07206) -> 1.00000000
(DB07206, DB07993) -> 1.00000000
(DB07993, DB07175) -> 1.00000000
(DB07175, DB07110) -> 1.00000000


#### Compute pairwise node similarity using the Sim-rank algorithm. 

In [46]:
print(net.simrank_similarity(I, source='DB07415', target='DB07281'))

0.9


In [47]:
print(net.simrank_similarity(I, source='P56817', target='DB07415'))

0.0


In [48]:
print(net.simrank_similarity(I, source='DB07175', target='DB07110'))

0.9


We can clearly see that the similarity scores are almost similar for this selected sub-graph nodes

#### Compare and contrast the results using the two similarity measures (Jaccard/cosine etc. and Sim-Rank). Also try to interpret the similarity results from the two similarity measures.

SimRank is a general similarity measure, based on a simple and intuitive graph-theoretic model. SimRank is applicable in any domain with object-to-object relationships, that measures similarity of the structural context in which objects occur, based on their relationships with other objects. SimRank is a similarity metric that says “two objects are considered to be similar if they are referenced by similar objects.” 

The Jaccard Similarity, also called the Jaccard Index or Jaccard Similarity Coefficient, is a classic measure of similarity between two sets that was introduced by Paul Jaccard in 1901. Given two sets, A and B, the Jaccard Similarity is defined as the size of the intersection of set A and set B (i.e. the number of common elements) over the size of the union of set A and set B (i.e. the number of unique elements).


So, for our graph, Jaccard similarity scores denote the common edges interaction between nodes and SimRank similarity gives the reference measures, which comes out to be very similar since we have considered a small sub-graph. For a large graph, scores might vary based on the scenarios. 